# Jupyter widgets
[Jupyter widgets](https://ipywidgets.readthedocs.io/en/latest/index.html) are visual components that enhance the interaction with a Jupyter notebook. In this notebook we test some of the widgets. 

In [1]:
import os
import numpy as np
from osgeo import gdal, osr, ogr
import matplotlib.pyplot as plt
from skimage import io
from skimage import exposure
from skimage.io import imsave, imread
#from rasterio.plot import show
from IPython.display import display
from ipywidgets import Checkbox, VBox, HBox, Layout, Text, Image
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import warnings
print("NumPy version: %s"%np.__version__)
print("GDAL version: %s"%gdal.__version__)
warnings.filterwarnings('ignore')

NumPy version: 1.26.4
GDAL version: 3.9.2


## Interact
It is a widget that returns the value of a function for each value of its arguments passed through the widget. The type of widget depends on the type of argument passed to the function. For example, if we pass an integer interval thw widget used will be an integer slider.

In [2]:
def f(x):
    return x * x

In [4]:
w = interact(f, x=(0,9));

interactive(children=(IntSlider(value=4, description='x', max=9), Output()), _dom_classes=('widget-interact',)…

In [5]:
x = w.widget.children[0].get_state('value')['value']
x

4

In [6]:
f(x)

16

## Slider

In [52]:
slider = widgets.IntSlider(value=0, min=0, max=10, step=1)
display(slider)

IntSlider(value=0, max=10)

In [53]:
current_value = slider.value
print('Slider value: {:d}'.format(current_value))

Slider value: 4


## Checkbox
Checkboxes are used to select multiple elements from a collection, e.g. images.

In [63]:
c = widgets.Checkbox(value=False, description='Check me')
display(c)

Checkbox(value=False, description='Check me')

In [65]:
c.value

True

In [142]:
img_path = 'images/'
img_list = []
for file in os.listdir(img_path):
    if file.endswith(".tif"):
        img_list.append(os.path.join(img_path, file))

In [143]:
img_list

['images/Building1.tif',
 'images/Building2.tif',
 'images/Building3.tif',
 'images/Building4.tif',
 'images/Building5.tif',
 'images/Building6.tif',
 'images/Building7.tif',
 'images/Building8.tif']

In [144]:
def f(delete):
    return delete

In [145]:
def browse_images(img_list):
    '''
    This function uses a widget to show and flag the images
    '''
    n = len(img_list)
    check_list = []
    for i in range(n):
        #check = interactive(f, delete=False, description='Check me');
        check = widgets.Checkbox(description='Remove')
        check_list.append(check)
        
    def view_image(i):
        img_or = io.imread(img_list[i])
        img = exposure.equalize_hist(img_or)
        plt.imshow(img, cmap=plt.cm.gray_r, interpolation='nearest')
        title = img_list[i][-10:-4]
        plt.title('Training: %s' % title)
        display(check_list[i])
        plt.show()
    
    interact(view_image, i = (0, n - 1))
    return check_list

In [146]:
check_list = browse_images(img_list)

interactive(children=(IntSlider(value=3, description='i', max=7), Output()), _dom_classes=('widget-interact',)…

In [148]:
for i in range(len(check_list)):
    print(check_list[i].value)

True
False
True
False
True
False
True
False
